In [1]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


df = pd.read_stata('./fwd_holding_merged_10q1-23q2.dta')

df = df[[
    'seriescode',
    'currency',
    'yq',
    'curPur_usd',
    'curSold_usd',
    'netassets'
]]

df['net_forward_sale'] = (df['curSold_usd'] - df['curPur_usd']) / df['netassets']

display(df)

,seriescode,currency,yq,curPur_usd,curSold_usd,netassets,net_forward_sale
0,S000000706,DKK,2009-10-01,4.717232e+06,4.617606e+06,8.010000e+07,-0.001244
1,S000000706,GBP,2009-10-01,1.876963e+07,2.718729e+07,8.010000e+07,0.105089
2,S000000706,JPY,2009-10-01,1.149187e+07,3.550935e+07,8.010000e+07,0.299844
3,S000000706,NOK,2009-10-01,2.988593e+06,8.168888e+06,8.010000e+07,0.064673
4,S000000706,SEK,2009-10-01,1.709673e+06,1.710846e+06,8.010000e+07,0.000015
...,...,...,...,...,...,...,...
64742,S000075280,CNY,2023-07-01,1.749745e+06,1.069824e+06,1.000000e+05,-6.799206
64743,S000075280,IDR,2023-07-01,3.070608e+05,0.000000e+00,1.000000e+05,-3.070607
64744,S000075280,KZT,2023-07-01,0.000000e+00,0.000000e+00,1.000000e+05,0.000000
64745,S000075280,PEN,2023-07-01,3.128826e+04,0.000000e+00,1.000000e+05,-0.312883


In [4]:
# Group by currency and date, keep net only
df_plot = df.groupby(['currency', 'yq'])[['net_forward_sale']].sum().reset_index()

display(df_plot[df_plot['currency'] == 'AED'])

,currency,yq,net_forward_sale
0,AED,2010-07-01,0.000000e+00
1,AED,2010-10-01,0.000000e+00
2,AED,2013-07-01,0.000000e+00
3,AED,2013-10-01,0.000000e+00
4,AED,2014-01-01,0.000000e+00
5,AED,2014-04-01,0.000000e+00
6,AED,2014-07-01,0.000000e+00
7,AED,2015-07-01,1.207412e-01
8,AED,2015-10-01,1.437004e-01
9,AED,2016-01-01,1.669940e-01


In [ ]:

def currency_plot(df_plot):
    # Convert 'yq' to a quarterly period string for the bar chart legend.
    df_plot['select_date'] = pd.to_datetime(df_plot['yq'], format='%Y%m%d').dt.to_period('Q').astype(str)
    df_plot = df_plot.sort_values('select_date')

    # Create the bar chart
    bar_fig = px.bar(
        df_plot,
        x="net_forward_sale",
        y="currency",
        color="select_date",
        orientation="h",
        title="Net Forward Sale Position by Date"
    )
    
    # Set each trace to be initially hidden in the plot (i.e., non-selected).
    for trace in bar_fig.data:
        trace.visible = "legendonly"

    bar_fig.update_layout(
        xaxis_title="", 
        yaxis_title="Currency",
        template="plotly_white"
    )
    

    # bar_fig.write_json("bar_chart.json")
    bar_fig.write_html("bar_chart.html", include_plotlyjs="cdn")


    app = dash.Dash(__name__)

    # Define the layout
    app.layout = html.Div([
        html.H1("", style={'textAlign': 'center'}),

        # Section 1: Bar Chart 
        html.Div([
            html.H2("Net Position by Currency Across Dates"),
            dcc.Graph(id='bar-chart', figure=bar_fig)
        ], style={'marginBottom': 50, 'width': '90%', 'margin': 'auto'}),

        # Section 2: Area Chart
        html.Div([
            html.H2("Net Position by Currency Over Time"),
            dcc.Dropdown(
                id='currency-dropdown',
                options=[{'label': cur, 'value': cur} for cur in sorted(df_plot['currency'].unique())],
                value=sorted(df_plot['currency'].unique()),  
                multi=True,
                placeholder="Select currencies"
            ),
            dcc.Graph(id='area-plot')
        ], style={'width': '90%', 'margin': 'auto'})
    ])

    # Callback to update
    @app.callback(
        Output('area-plot', 'figure'),
        [Input('currency-dropdown', 'value')]
    )
    def update_area_chart(selected_currencies):
        filtered_df = df_plot[df_plot['currency'].isin(selected_currencies)] if selected_currencies else df_plot.iloc[0:0]
        area_fig = px.area(filtered_df, x="yq", y="net_forward_sale", color="currency")
        area_fig.update_layout(
            xaxis_title="",
            yaxis_title="Net Forward Sale Position by Currency",
            template="plotly_white"
        )
        
        '''
        # Optional: Also set each trace to be initially hidden
        for trace in area_fig.data:
            trace.visible = "legendonly"
        '''
        # area_fig.write_json("area_chart.json")
        area_fig.write_html("area_chart.html", include_plotlyjs="cdn")
        return area_fig

    if __name__ == '__main__':
        app.run_server(debug=True)



currency_plot(df_plot)

